In [ ]:
import numpy as np
from pyscf import gto, scf, ao2mo, ci, cc, fci
from pyscf.tools import fcidump

# =============================================================================
# 1. Read FCIDUMP file and setup molecular system
# =============================================================================
print("=" * 60)
print("Reading FCIDUMP file and setting up system")
print("=" * 60)

fcidump_file = '../FCIDUMP/N2_sto3g_2.50.FCIDUMP'
data = fcidump.read(fcidump_file)

# Extract integrals and system parameters
h1e = data['H1']          # One-electron integrals
h2e = data['H2']          # Two-electron integrals
n_elec = data['NELEC']    # Total number of electrons
n_orbs = data['NORB']     # Number of orbitals
e_nuc = data['ECORE']     # Nuclear repulsion energy

print(f"Number of orbitals: {n_orbs}")
print(f"Number of electrons: {n_elec}")
print(f"Nuclear repulsion energy: {e_nuc:.8f} Ha\n")

# =============================================================================
# 2. Setup molecule and RHF calculation using FCIDUMP integrals
# =============================================================================
print("=" * 60)
print("Setting up RHF calculation")
print("=" * 60)

# Create a minimal molecule object
mol = gto.Mole()
mol.nelectron = n_elec
mol.spin = 0  # Singlet state (Ms = 0)
mol.build()

# Initialize RHF object and override with FCIDUMP integrals
mf = scf.RHF(mol)
mf.energy_nuc = lambda *args: e_nuc
mf.get_hcore = lambda *args: h1e
mf.get_ovlp = lambda *args: np.identity(n_orbs)  # MO basis has identity overlap
mf._eri = ao2mo.restore(1, h2e, n_orbs)  # Restore 8-fold symmetry

# Run HF calculation (should converge in one iteration)
mf.kernel()
print(f"Hartree-Fock Energy: {mf.e_tot:.8f} Ha\n")

# =============================================================================
# 3. FCI calculation without spin constraint (problematic)
# =============================================================================
print("=" * 60)
print("FCI Calculation #1: Without spin constraint (Ms conserved only)")
print("=" * 60)

neleca = n_elec // 2
nelecb = n_elec - neleca
nelec_tuple = (neleca, nelecb)

# Use direct_spin1 solver (conserves Ms but not S)
myfci_unconstrained = fci.direct_spin1.FCI()
myfci_unconstrained.nroots = 5

# Calculate multiple roots
fci_energies_uncon, fci_vecs_uncon = myfci_unconstrained.kernel(
    h1e, h2e, n_orbs, nelec_tuple
)

print("Found roots (may contain mixed spin states):")
for i, vec in enumerate(fci_vecs_uncon):
    s2 = myfci_unconstrained.spin_square(vec, n_orbs, nelec_tuple)[0]
    total_energy = fci_energies_uncon[i] + e_nuc
    print(f"  Root {i}: E_total = {total_energy:.8f} Ha, <S²> = {s2:.4f}")
print("Note: S=0 expects <S²>=0.0, S=1 expects <S²>=2.0\n")

# =============================================================================
# 4. FCI calculation with singlet constraint (correct)
# =============================================================================
print("=" * 60)
print("FCI Calculation #2: With singlet (S=0) constraint")
print("=" * 60)

# Use fix_spin to enforce pure singlet states (ss=0 means S²=0)
myfci_singlet = fci.addons.fix_spin(fci.FCI(mol), ss=0)
myfci_singlet.nroots = 5

# Calculate singlet-only roots
fci_energies_singlet, fci_vecs_singlet = myfci_singlet.kernel(
    h1e, h2e, n_orbs, nelec_tuple
)

print("Found roots (enforced singlets):")
for i, vec in enumerate(fci_vecs_singlet):
    s2 = fci.spin_op.spin_square(vec, n_orbs, nelec_tuple)[0]
    total_energy = fci_energies_singlet[i] + e_nuc
    print(f"  Root {i}: E_total = {total_energy:.8f} Ha, <S²> = {s2:.4f}")

# =============================================================================
# 6. Summary and comparison
# =============================================================================
print("\n" + "=" * 60)
print("SUMMARY: Energy Comparison")
print("=" * 60)

print(f"Hartree-Fock:                    {mf.e_tot:.8f} Ha")
print(f"FCI (unconstrained, lowest):     {fci_energies_uncon[0] + e_nuc:.8f} Ha")
print(f"FCI (singlet, ground state):     {fci_energies_singlet[0] + e_nuc:.8f} Ha")

print("\n" + "=" * 60)
print("CONCLUSION")
print("=" * 60)
print("The singlet-constrained FCI gives the correct ground state energy")
print("for exact diagonalization in the singlet subspace.")
print("=" * 60)

In [ ]:
# Copyright 2025 - All rights reserved.
# SPDX-License-Identifier: Apache-2.0

"""
Diatomic molecule potential energy surface calculator using FCI.

Computes ground state energies for singlet and triplet spin states across
a range of bond distances with adaptive point distribution.

Author: Zheng (Alex) Che
Date: January 2025
"""

import csv
from dataclasses import dataclass
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
from pyscf import fci, gto, scf

# ============================================================================
# User Configuration
# ============================================================================

@dataclass
class ScanConfig:
    """Configuration for PES scan."""
    atom_symbol: str = "B"        # Element symbol
    basis: str = "sto-3g"         # Basis set
    ref_distance: float = 1.59    # Reference bond length (Angstrom)
    total_points: int = 35        # Number of scan points
    num_roots: int = 20           # FCI roots per spin state
  
    # Scan range relative to reference
    range_start: float = 0.8      # Multiplier for min distance
    range_end: float = 3.0        # Multiplier for max distance
  
    # Point distribution (higher = denser near reference)
    density_power: float = 2.0
    points_before_ref: int = 10

config = ScanConfig()

# ============================================================================
# Point Generation
# ============================================================================

def generate_scan_points(cfg: ScanConfig) -> np.ndarray:
    """
    Generate non-uniform bond distance points.
  
    Creates denser sampling near reference distance using power-law spacing.
  
    Args:
        cfg: Configuration parameters
      
    Returns:
        Sorted array of unique distances rounded to 2 decimals
    """
    if cfg.total_points <= cfg.points_before_ref:
        raise ValueError(f"total_points must exceed {cfg.points_before_ref}")
  
    # Uniform points before reference
    d_min = cfg.range_start * cfg.ref_distance
    d_max = cfg.range_end * cfg.ref_distance
    before = np.linspace(d_min, cfg.ref_distance, cfg.points_before_ref, endpoint=False)
  
    # Power-law distributed points after reference
    n_after = cfg.total_points - cfg.points_before_ref
    linear = np.linspace(0, 1, n_after)
    scaled = linear ** cfg.density_power
    after = cfg.ref_distance + scaled * (d_max - cfg.ref_distance)
  
    return np.unique(np.round(np.concatenate([before, after]), 2))


# ============================================================================
# FCI Calculation
# ============================================================================

def compute_fci_energy(distance: float, spin: int, cfg: ScanConfig) -> Tuple[float, float, float]:
    """
    Compute FCI ground state energy at given geometry.
  
    Args:
        distance: Bond length (Angstrom)
        spin: 2S value (0=singlet, 2=triplet)
        cfg: Calculation parameters
      
    Returns:
        (total_energy, S², nuclear_repulsion) in Hartree
    """
    # Build molecule
    mol = gto.Mole()
    mol.atom = f'{cfg.atom_symbol} 0 0 0; {cfg.atom_symbol} 0 0 {distance}'
    mol.basis = cfg.basis
    mol.spin = spin
    mol.build()
  
    # Mean-field reference
    mf = (scf.RHF if spin == 0 else scf.ROHF)(mol)
    mf.kernel()
  
    # Transform to MO basis
    h1e = mf.mo_coeff.T @ mf.get_hcore() @ mf.mo_coeff
    eri = mol.intor('int2e')
    h2e = np.einsum('pi,qj,pqrs,rk,sl->ijkl', 
                    mf.mo_coeff, mf.mo_coeff, eri,
                    mf.mo_coeff, mf.mo_coeff, optimize=True)
  
    # Setup FCI with spin constraint
    n_elec = mol.nelectron
    nelec_tuple = ((n_elec + spin) // 2, (n_elec - spin) // 2)
  
    solver = fci.addons.fix_spin(fci.FCI(mol), ss=spin)
    solver.nroots = cfg.num_roots
  
    # Solve and extract ground state
    energies, vecs = solver.kernel(h1e, h2e, mol.nao, nelec_tuple)
    e_elec = energies[0]
    s_squared = fci.spin_op.spin_square(vecs[0], mol.nao, nelec_tuple)[0]
  
    return e_elec + mol.energy_nuc(), s_squared, mol.energy_nuc()


# ============================================================================
# Main Scan
# ============================================================================

def run_pes_scan(cfg: ScanConfig):
    """Execute full PES scan and generate outputs."""
  
    print("=" * 70)
    print(f"{cfg.atom_symbol}₂ Potential Energy Surface Scan")
    print("=" * 70)
    print(f"Basis: {cfg.basis} | Points: {cfg.total_points} | Reference: {cfg.ref_distance} Å")
    print("=" * 70)
  
    distances = generate_scan_points(cfg)
  
    # Storage arrays
    E_singlet, E_triplet = [], []
    S2_singlet, S2_triplet = [], []
  
    print("\nScanning geometries...")
    for i, d in enumerate(distances):
        print(f"[{i+1}/{len(distances)}] d = {d:.3f} Å", end=" ")
      
        # Singlet
        e_s, s2_s, nuc_s = compute_fci_energy(d, spin=0, cfg=cfg)
        E_singlet.append(e_s)
        S2_singlet.append(s2_s)
        print(f"| S: {e_s:.6f} Ha", end=" ")
      
        # Triplet
        e_t, s2_t, nuc_t = compute_fci_energy(d, spin=2, cfg=cfg)
        E_triplet.append(e_t)
        S2_triplet.append(s2_t)
        print(f"| T: {e_t:.6f} Ha")
  
    # Convert to arrays
    data = {
        'distances': distances,
        'E_singlet': np.array(E_singlet),
        'E_triplet': np.array(E_triplet),
        'S2_singlet': np.array(S2_singlet),
        'S2_triplet': np.array(S2_triplet)
    }
  
    # Reference energies (singlet at ref_distance)
    ref_idx = np.argmin(np.abs(distances - cfg.ref_distance))
    E_ref = data['E_singlet'][ref_idx]
  
    # Relative energies (eV)
    Ha_to_eV = 27.211396
    data['rel_singlet'] = (data['E_singlet'] - E_ref) * Ha_to_eV
    data['rel_triplet'] = (data['E_triplet'] - E_ref) * Ha_to_eV
  
    return data, ref_idx


# ============================================================================
# Output Generation
# ============================================================================

def save_csv(data: dict, filename: str, atom: str):
    """Save PES data to CSV file."""
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([f'{atom}-{atom}_Distance_Angstrom', 
                        'Singlet_Energy_Hartree', 
                        'Triplet_Energy_Hartree'])
        for i in range(len(data['distances'])):
            writer.writerow([f"{data['distances'][i]:.6f}",
                           f"{data['E_singlet'][i]:.10f}",
                           f"{data['E_triplet'][i]:.10f}"])
    print(f"\nData saved: {filename}")


def plot_pes(data: dict, ref_idx: int, cfg: ScanConfig):
    """Generate publication-quality PES plot."""
  
    plt.rcParams.update({
        'font.size': 11,
        'axes.labelsize': 12,
        'lines.linewidth': 2.5,
        'legend.fontsize': 11
    })
  
    fig, ax = plt.subplots(figsize=(10, 7))
  
    d = data['distances']
    d_ref = d[ref_idx]
  
    # Plot curves
    ax.plot(d, data['rel_singlet'], 'o-', color='steelblue', 
            markersize=6, markeredgecolor='white', markeredgewidth=1,
            label='Singlet (S=0)', zorder=3)
    ax.plot(d, data['rel_triplet'], 's-', color='crimson',
            markersize=6, markeredgecolor='white', markeredgewidth=1,
            label='Triplet (S=1)', zorder=3)
  
    # Reference line and markers
    ax.axvline(d_ref, color='gray', linestyle='--', linewidth=1.5,
              alpha=0.5, label=f'Ref: {d_ref} Å')
    ax.plot(d_ref, 0.0, 'o', color='steelblue', markersize=12,
           markeredgecolor='black', markeredgewidth=1.5, zorder=4)
    ax.plot(d_ref, data['rel_triplet'][ref_idx], 's', color='crimson',
           markersize=12, markeredgecolor='black', markeredgewidth=1.5, zorder=4)
  
    # Annotations
    s2_s = data['S2_singlet'][ref_idx]
    s2_t = data['S2_triplet'][ref_idx]
    e_t = data['rel_triplet'][ref_idx]
  
    ax.annotate(f'S: 0.00 eV\n⟨S²⟩={s2_s:.3f}',
               xy=(d_ref, 0), xytext=(d_ref+0.15, 0),
               bbox=dict(boxstyle='round,pad=0.3', fc='white', 
                        ec='steelblue', alpha=0.8))
    ax.annotate(f'T: {e_t:.2f} eV\n⟨S²⟩={s2_t:.3f}',
               xy=(d_ref, e_t), xytext=(d_ref+0.15, e_t),
               bbox=dict(boxstyle='round,pad=0.3', fc='white',
                        ec='crimson', alpha=0.8))
  
    # Formatting
    ax.set_xlabel(f'{cfg.atom_symbol}-{cfg.atom_symbol} Distance (Å)', fontsize=13)
    ax.set_ylabel(f'Relative Energy (eV)\n[Ref: Singlet @ {d_ref} Å]', fontsize=13)
    ax.set_title(f'{cfg.atom_symbol}₂ Dissociation PES (FCI/{cfg.basis})',
                fontsize=15, fontweight='bold')
    ax.legend(loc='upper right', framealpha=0.95)
    ax.grid(True, alpha=0.3)
  
    # Auto y-limits with margin
    y_all = np.concatenate([data['rel_singlet'], data['rel_triplet']])
    margin = (y_all.max() - y_all.min()) * 0.1
    ax.set_ylim(y_all.min() - margin, y_all.max() + margin)
  
    plt.tight_layout()
    filename = f'{cfg.atom_symbol}2_PES_FCI.pdf'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Plot saved: {filename}")
    plt.show()


def print_summary(data: dict, cfg: ScanConfig):
    """Print analysis summary."""
  
    print("\n" + "=" * 70)
    print("ANALYSIS")
    print("=" * 70)
  
    # Find minima
    idx_s = np.argmin(data['E_singlet'])
    idx_t = np.argmin(data['E_triplet'])
  
    print(f"\nSinglet minimum:")
    print(f"  d = {data['distances'][idx_s]:.3f} Å")
    print(f"  E = {data['E_singlet'][idx_s]:.6f} Ha ({data['rel_singlet'][idx_s]:.3f} eV)")
    print(f"  ⟨S²⟩ = {data['S2_singlet'][idx_s]:.4f}")
  
    print(f"\nTriplet minimum:")
    print(f"  d = {data['distances'][idx_t]:.3f} Å")
    print(f"  E = {data['E_triplet'][idx_t]:.6f} Ha ({data['rel_triplet'][idx_t]:.3f} eV)")
    print(f"  ⟨S²⟩ = {data['S2_triplet'][idx_t]:.4f}")
  
    # Crossing detection
    diff = data['rel_triplet'] - data['rel_singlet']
    crossings = np.where(np.diff(np.sign(diff)))[0]
    if len(crossings) > 0:
        d_cross = (data['distances'][crossings[0]] + data['distances'][crossings[0]+1]) / 2
        print(f"\nApproximate S-T crossing: ~{d_cross:.3f} Å")
    else:
        print(f"\nNo S-T crossing in scan range")
  
    print("=" * 70)


# ============================================================================
# Execute
# ============================================================================

if __name__ == "__main__":
    data, ref_idx = run_pes_scan(config)
    save_csv(data, f'{config.atom_symbol}2_FCI_PES.csv', config.atom_symbol)
    plot_pes(data, ref_idx, config)
    print_summary(data, config)